In [1]:
## Polecenia do instalacji pakietów
# conda install -c conda-forge tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
data_AUG = pd.read_csv("apartments_pl_2023_08.csv")
data_SEP = pd.read_csv("apartments_pl_2023_09.csv")
data_OCT = pd.read_csv("apartments_pl_2023_10.csv")

# Analiza danych

Wyniki analizy danych (specyficzne dla konkretnego zbioru danych) standardowo powinny zawierać przynajmniej następujące elementy:
* liczba atrybutów i typy ich wartości
* liczba różnych wartości atrybutów, które nie są ciągłe (ze skończonego, niewielkiego zakresu)
* atrybut decyzyjny - charakterystyka
* brakujące wartości (z podziałem na obiekty i w ramach poszczególnych atrybutów warunkowych)
* duplikaty
* liczebności przykładów z poszczególnych klas
* współczynnik IR (imbalanced ratio)
* mapa korelacji atrybutów
* weryfikacja zasadności pozostawienia/usunięcia poszczególnych atrybutów na podstawie wiedzy dziedzinowej
* analiza (komentarze dodawane na bieżąco w notatniku)
* wnioski

In [3]:
print("data_AUG length: ", len(data_AUG))
print("data_SEP length: ", len(data_SEP))
print("data_OCT length: ", len(data_OCT))

data_AUG length:  18905
data_SEP length:  16997
data_OCT length:  16690


In [4]:
data_AUG.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,f8524536d4b09a0c8ccc0197ec9d7bde,szczecin,blockOfFlats,63.00,3.0,4.0,10.0,1980.0,53.378933,14.625296,...,0.413,condominium,concreteSlab,NaN,yes,yes,yes,no,yes,415000
1,accbe77d4b360fea9735f138a50608dd,szczecin,blockOfFlats,36.00,2.0,8.0,10.0,NaN,53.442692,14.559690,...,0.205,cooperative,concreteSlab,NaN,no,yes,yes,no,yes,395995
2,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
3,0a68cd14c44ec5140143ece75d739535,szczecin,tenement,87.60,3.0,2.0,3.0,NaN,53.435100,14.532900,...,0.087,condominium,brick,NaN,yes,yes,no,no,yes,640000
4,f66320e153c2441edc0fe293b54c8aeb,szczecin,blockOfFlats,66.00,3.0,1.0,3.0,NaN,53.410278,14.503611,...,0.514,condominium,NaN,NaN,no,no,no,no,no,759000


In [5]:
data_SEP.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,71de9eb8a92ac12d7c94315228b70cfb,szczecin,tenement,123.45,4.0,2.0,3.0,NaN,53.432586,14.535685,...,0.321,condominium,brick,NaN,no,yes,no,no,yes,799000
1,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.073,cooperative,NaN,NaN,no,no,yes,no,no,420000
2,3b8819c6c9ffd55adfddfd5bafcc844d,szczecin,tenement,50.00,2.0,NaN,1.0,NaN,53.364400,14.660900,...,0.399,condominium,brick,NaN,yes,no,no,no,no,350000
3,708ccd264180315a4cfe6324ed4861d0,szczecin,blockOfFlats,55.19,2.0,1.0,2.0,2013.0,53.480364,14.541494,...,0.183,condominium,brick,premium,yes,yes,no,yes,yes,607000
4,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000


In [6]:
data_OCT.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.073,cooperative,NaN,NaN,no,no,yes,no,no,420000
1,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
2,ede1c0d953ff85f3cd9676d39ca8f62d,szczecin,blockOfFlats,37.00,2.0,8.0,10.0,1970.0,53.442692,14.559690,...,0.205,cooperative,concreteSlab,NaN,no,yes,yes,yes,yes,429995
3,6c807c6c86a74c57a3fd4445a1f49a46,szczecin,apartmentBuilding,48.00,2.0,4.0,6.0,NaN,53.435461,14.570436,...,0.177,condominium,brick,NaN,no,yes,no,no,no,550000
4,bb14c9a540275da73f6cb6ed42e5c26c,szczecin,apartmentBuilding,55.19,2.0,1.0,2.0,2013.0,53.482094,14.542574,...,0.043,condominium,brick,premium,yes,yes,no,yes,yes,607000


In [7]:
# złączenie plików z danymi
data_ALL = pd.concat([data_AUG, data_SEP, data_OCT], ignore_index=True)

In [8]:
data_ALL.sample(10)

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
20605,2108124de4eab00310961d1c240798b9,krakow,blockOfFlats,28.10,1.0,3.0,3.0,2023.0,50.072773,19.935873,...,0.118,condominium,brick,NaN,no,no,no,no,no,477700
30905,343560b1e21e4a748ff341d346246198,warszawa,tenement,88.00,4.0,4.0,4.0,1924.0,52.219720,20.980980,...,0.191,condominium,brick,NaN,no,no,no,no,yes,1500000
31960,54f8843e6ab26432b546c2da83ee589a,warszawa,apartmentBuilding,72.36,3.0,1.0,3.0,2004.0,52.199600,20.932200,...,0.381,condominium,brick,NaN,no,no,yes,no,no,899000
9611,4efc8f37a7078b72fa872a2fd935a723,lodz,apartmentBuilding,88.00,4.0,3.0,3.0,2012.0,51.750560,19.574716,...,0.287,condominium,brick,premium,no,yes,yes,yes,yes,749000
7542,4eeacf08550413e5d9452e669f1cd44f,wroclaw,blockOfFlats,62.00,3.0,4.0,4.0,1980.0,51.097899,16.993617,...,0.367,condominium,concreteSlab,NaN,no,yes,no,no,yes,580000
44829,b8fec4c53f197804057d8a2225e6e459,lodz,apartmentBuilding,76.61,3.0,5.0,5.0,2023.0,51.765100,19.475800,...,0.314,condominium,brick,NaN,no,yes,yes,no,no,682595
33621,daf42e9baba63a941e3b5a9956f4f806,warszawa,apartmentBuilding,53.68,2.0,4.0,5.0,NaN,52.256400,20.930800,...,0.244,condominium,brick,NaN,no,yes,yes,yes,no,767000
47671,26ac6d6149ae50b62c40e0797c5a84eb,warszawa,NaN,45.31,2.0,NaN,3.0,2019.0,52.328988,21.007679,...,1.465,condominium,NaN,premium,no,no,no,no,no,610000
12304,172cae8acb5b1ed7ba6354577f901e6a,lublin,tenement,64.40,2.0,2.0,3.0,1900.0,51.245866,22.574293,...,0.403,condominium,brick,premium,no,no,no,no,yes,647900
14961,dad86bd2fee906a0eb81a142ed4affa0,warszawa,NaN,52.00,3.0,2.0,2.0,2016.0,52.328988,21.007679,...,1.465,condominium,brick,premium,yes,no,yes,yes,no,649000


## Liczba atrybutów i typy ich wartości

In [9]:
data_SEP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16997 entries, 0 to 16996
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    16997 non-null  object 
 1   city                  16997 non-null  object 
 2   type                  13171 non-null  object 
 3   squareMeters          16997 non-null  float64
 4   rooms                 16997 non-null  float64
 5   floor                 13954 non-null  float64
 6   floorCount            16751 non-null  float64
 7   buildYear             14048 non-null  float64
 8   latitude              16997 non-null  float64
 9   longitude             16997 non-null  float64
 10  centreDistance        16997 non-null  float64
 11  poiCount              16997 non-null  float64
 12  schoolDistance        16982 non-null  float64
 13  clinicDistance        16918 non-null  float64
 14  postOfficeDistance    16974 non-null  float64
 15  kindergartenDistanc

### Sierpień

In [10]:
data_AUG.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,18905.000000,18905.000000,15467.000000,18643.000000,15634.000000,18905.000000,18905.000000,18905.000000,18905.000000,18891.000000,18817.000000,18880.000000,18892.000000,18847.000000,18340.000000,18878.000000,1.890500e+04
mean,58.509976,2.676752,3.339820,5.279783,1984.551043,52.020411,19.516394,4.398444,20.099656,0.414897,1.002196,0.518748,0.371033,0.356646,1.460047,0.364134,6.925592e+05
std,21.060168,0.903619,2.549141,3.333666,33.767929,1.353998,1.769053,2.857387,23.513874,0.472782,0.913721,0.506783,0.452338,0.475884,1.102847,0.469334,3.443890e+05
min,25.000000,1.000000,1.000000,1.000000,1850.000000,49.979110,14.447127,0.020000,0.000000,0.004000,0.005000,0.001000,0.004000,0.001000,0.006000,0.003000,1.500000e+05
25%,44.200000,2.000000,2.000000,3.000000,1967.000000,51.109100,18.536220,2.070000,7.000000,0.175000,0.367000,0.237000,0.158000,0.118500,0.592000,0.145000,4.588000e+05
50%,54.400000,3.000000,3.000000,4.000000,1990.000000,52.193604,19.908944,4.010000,13.000000,0.288000,0.700000,0.387000,0.264000,0.235000,1.140000,0.239000,6.200000e+05
75%,68.200000,3.000000,4.000000,6.000000,2014.000000,52.412561,20.992010,6.180000,23.000000,0.466500,1.286000,0.621000,0.415000,0.417000,2.107000,0.405000,8.300000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.583210,23.207128,16.940000,208.000000,4.818000,4.996000,4.967000,4.960000,4.985000,4.998000,4.992000,2.500000e+06


### Wrzesień

In [11]:
data_SEP.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,16997.000000,16997.000000,13954.000000,16751.000000,14048.000000,16997.000000,16997.000000,16997.000000,16997.000000,16982.000000,16918.000000,16974.000000,16983.000000,16954.000000,16495.000000,16974.000000,1.699700e+04
mean,59.033912,2.689357,3.327576,5.237538,1984.351936,52.066227,19.528680,4.347108,20.345590,0.418712,0.988139,0.524299,0.378864,0.359300,1.451509,0.370333,7.080972e+05
std,21.362743,0.916103,2.532150,3.305643,34.041196,1.331268,1.790088,2.893583,23.710083,0.482618,0.911026,0.516412,0.467429,0.487905,1.109576,0.483060,3.519888e+05
min,25.000000,1.000000,1.000000,1.000000,1851.000000,49.979110,14.447127,0.020000,0.000000,0.004000,0.005000,0.003000,0.002000,0.001000,0.007000,0.001000,1.549000e+05
25%,44.500000,2.000000,2.000000,3.000000,1965.000000,51.120355,18.529520,1.960000,6.000000,0.175000,0.359000,0.237000,0.158000,0.116000,0.583000,0.146000,4.650000e+05
50%,55.190000,3.000000,3.000000,4.000000,1991.000000,52.197569,19.903611,3.930000,13.000000,0.292000,0.681000,0.391000,0.268000,0.234000,1.122000,0.240000,6.390000e+05
75%,69.000000,3.000000,4.000000,6.000000,2014.000000,52.450200,20.998580,6.150000,24.000000,0.469000,1.266000,0.629000,0.423000,0.418000,2.077500,0.411000,8.500000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.606460,23.207128,16.920000,208.000000,4.818000,4.989000,4.905000,4.960000,4.985000,5.000000,4.861000,2.500000e+06


### Październik

In [12]:
data_OCT.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,16690.000000,16690.000000,13644.000000,16448.000000,13785.000000,16690.000000,16690.000000,16690.000000,16690.000000,16669.000000,16605.000000,16665.000000,16668.000000,16637.000000,16185.000000,16661.000000,1.669000e+04
mean,58.985279,2.680228,3.268030,5.162755,1984.573957,52.047413,19.489039,4.328341,20.462912,0.417843,0.978049,0.526101,0.376593,0.357309,1.444789,0.365081,7.235462e+05
std,21.347723,0.911191,2.455211,3.201862,34.164553,1.331076,1.779749,2.879183,24.104101,0.480638,0.905555,0.515941,0.463518,0.483880,1.105216,0.476461,3.521544e+05
min,25.000000,1.000000,1.000000,1.000000,1851.000000,49.981357,14.447127,0.020000,0.000000,0.005000,0.001000,0.003000,0.002000,0.001000,0.007000,0.001000,1.526200e+05
25%,44.402500,2.000000,2.000000,3.000000,1965.000000,51.114060,18.512167,1.970000,7.000000,0.175000,0.355000,0.241000,0.157000,0.117000,0.586000,0.145000,4.800000e+05
50%,55.000000,3.000000,3.000000,4.000000,1992.000000,52.195677,19.894169,3.920000,13.000000,0.292000,0.674000,0.395000,0.266000,0.235000,1.119000,0.242000,6.590000e+05
75%,69.000000,3.000000,4.000000,6.000000,2015.000000,52.440733,20.994512,6.090000,24.000000,0.468000,1.246000,0.629000,0.423000,0.419000,2.051000,0.407000,8.700000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.606460,23.195701,16.920000,208.000000,4.818000,4.989000,4.905000,4.960000,4.985000,5.000000,4.861000,2.500000e+06


### Dane złączone

In [13]:
data_ALL.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,52592.000000,52592.000000,43065.000000,51842.000000,43467.000000,52592.000000,52592.000000,52592.000000,52592.000000,52542.000000,52340.000000,52519.000000,52543.000000,52438.000000,51020.000000,52513.000000,5.259200e+04
mean,58.830142,2.681929,3.313108,5.229004,1984.493961,52.043787,19.511684,4.359606,20.294417,0.417065,0.989992,0.522876,0.375328,0.357714,1.452446,0.366438,7.074146e+05
std,21.250642,0.910070,2.514338,3.283571,33.981805,1.339533,1.779307,2.876144,23.765901,0.478467,0.910303,0.512821,0.460811,0.482326,1.105774,0.476065,3.495542e+05
min,25.000000,1.000000,1.000000,1.000000,1850.000000,49.979110,14.447127,0.020000,0.000000,0.004000,0.001000,0.001000,0.002000,0.001000,0.006000,0.001000,1.500000e+05
25%,44.360000,2.000000,2.000000,3.000000,1965.000000,51.114060,18.527700,2.010000,7.000000,0.175000,0.360000,0.238000,0.158000,0.117000,0.587000,0.145000,4.680000e+05
50%,55.000000,3.000000,3.000000,4.000000,1991.000000,52.195252,19.902880,3.960000,13.000000,0.290000,0.685000,0.390000,0.266000,0.235000,1.124000,0.240000,6.390000e+05
75%,69.000000,3.000000,4.000000,6.000000,2014.000000,52.436800,20.994660,6.140000,24.000000,0.468000,1.270000,0.628000,0.420000,0.418000,2.076000,0.407000,8.500000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.606460,23.207128,16.940000,208.000000,4.818000,4.996000,4.967000,4.960000,4.985000,5.000000,4.992000,2.500000e+06


## Liczba różnych wartości atrybutów, które nie są ciągłe (ze skończonego, niewielkiego zakresu)

In [14]:
def get_unique_count(data_frame):
    temp = []
    for column in data_frame.columns.values:
        temp.append([column, len(data_frame[column].unique())])
        
    data_SEP_unique = pd.DataFrame(np.array(temp), columns=["attribute", "unique"])
    return data_SEP_unique

In [15]:
print("Liczba unikalnych wartości dla poszczególnych kolumn - Sierpień")
get_unique_count(data_AUG)

Liczba unikalnych wartości dla poszczególnych kolumn - Sierpień


,attribute,unique
0,id,18905
1,city,15
2,type,4
3,squareMeters,4214
4,rooms,6
5,floor,25
6,floorCount,30
7,buildYear,156
8,latitude,13762
9,longitude,14016


In [16]:
print("Liczba unikalnych wartości dla poszczególnych kolumn - Wrzesień")
get_unique_count(data_SEP)

Liczba unikalnych wartości dla poszczególnych kolumn - Wrzesień


,attribute,unique
0,id,16997
1,city,15
2,type,4
3,squareMeters,4042
4,rooms,6
5,floor,24
6,floorCount,30
7,buildYear,154
8,latitude,12493
9,longitude,12695


In [17]:
print("Liczba unikalnych wartości dla poszczególnych kolumn - Październik")
get_unique_count(data_OCT)

Liczba unikalnych wartości dla poszczególnych kolumn - Październik


,attribute,unique
0,id,16690
1,city,15
2,type,4
3,squareMeters,4068
4,rooms,6
5,floor,24
6,floorCount,30
7,buildYear,152
8,latitude,12340
9,longitude,12513


In [18]:
print("Liczba unikalnych wartości dla poszczególnych kolumn - wszystkie dane")
get_unique_count(data_ALL)

Liczba unikalnych wartości dla poszczególnych kolumn - wszystkie dane


,attribute,unique
0,id,32027
1,city,15
2,type,4
3,squareMeters,5175
4,rooms,6
5,floor,25
6,floorCount,30
7,buildYear,158
8,latitude,20723
9,longitude,21215


In [19]:
# TODO: prawdopodobnie wszystko co ma liczność powyżej setek będzie tutaj do wycięcia, 
# bo mają być nieciągłe atrybuty. Będzie można po prostu zrobić tablicę z nazwami kolumn i 
# wyświetlić tylko te, które są potrzebne

## Atrybut decyzyjny - charakterystyka

In [20]:
price_AUG = data_AUG["price"].rename("price_AUG")
price_SEP = data_SEP["price"].rename("price_SEP")
price_OCT = data_OCT["price"].rename("price_OCT")

In [21]:
# porównanie statystyk cen z poszczególnych miesięcy
pd.concat([
        price_AUG.describe().map(int), 
        price_SEP.describe().map(int), 
        price_OCT.describe().map(int)
    ],
    axis=1
)

,price_AUG,price_SEP,price_OCT
count,18905,16997,16690
mean,692559,708097,723546
std,344389,351988,352154
min,150000,154900,152620
25%,458800,465000,480000
50%,620000,639000,659000
75%,830000,850000,870000
max,2500000,2500000,2500000


## Brakujące wartości (z podziałem na obiekty i w ramach poszczególnych atrybutów warunkowych)

In [22]:
# TODO: zrozumieć część polecenia w nawiasie

In [23]:
def print_missing_values(data_frame):
    for column in data_frame.columns.values:
        print("\t", data_frame[column].isna().sum(), "\t", column)

In [24]:
print("N/A values for each column (August): ")
print_missing_values(data_AUG)

N/A values for each column (August): 
	 0 	 id
	 0 	 city
	 4039 	 type
	 0 	 squareMeters
	 0 	 rooms
	 3438 	 floor
	 262 	 floorCount
	 3271 	 buildYear
	 0 	 latitude
	 0 	 longitude
	 0 	 centreDistance
	 0 	 poiCount
	 14 	 schoolDistance
	 88 	 clinicDistance
	 25 	 postOfficeDistance
	 13 	 kindergartenDistance
	 58 	 restaurantDistance
	 565 	 collegeDistance
	 27 	 pharmacyDistance
	 0 	 ownership
	 7387 	 buildingMaterial
	 14344 	 condition
	 0 	 hasParkingSpace
	 0 	 hasBalcony
	 926 	 hasElevator
	 0 	 hasSecurity
	 0 	 hasStorageRoom
	 0 	 price


In [25]:
print("N/A values for each column (September): ")
print_missing_values(data_SEP)

N/A values for each column (September): 
	 0 	 id
	 0 	 city
	 3826 	 type
	 0 	 squareMeters
	 0 	 rooms
	 3043 	 floor
	 246 	 floorCount
	 2949 	 buildYear
	 0 	 latitude
	 0 	 longitude
	 0 	 centreDistance
	 0 	 poiCount
	 15 	 schoolDistance
	 79 	 clinicDistance
	 23 	 postOfficeDistance
	 14 	 kindergartenDistance
	 43 	 restaurantDistance
	 502 	 collegeDistance
	 23 	 pharmacyDistance
	 0 	 ownership
	 6635 	 buildingMaterial
	 12958 	 condition
	 0 	 hasParkingSpace
	 0 	 hasBalcony
	 868 	 hasElevator
	 0 	 hasSecurity
	 0 	 hasStorageRoom
	 0 	 price


In [26]:
print("N/A values for each column (October): ")
print_missing_values(data_OCT)

N/A values for each column (October): 
	 0 	 id
	 0 	 city
	 3785 	 type
	 0 	 squareMeters
	 0 	 rooms
	 3046 	 floor
	 242 	 floorCount
	 2905 	 buildYear
	 0 	 latitude
	 0 	 longitude
	 0 	 centreDistance
	 0 	 poiCount
	 21 	 schoolDistance
	 85 	 clinicDistance
	 25 	 postOfficeDistance
	 22 	 kindergartenDistance
	 53 	 restaurantDistance
	 505 	 collegeDistance
	 29 	 pharmacyDistance
	 0 	 ownership
	 6483 	 buildingMaterial
	 12517 	 condition
	 0 	 hasParkingSpace
	 0 	 hasBalcony
	 894 	 hasElevator
	 0 	 hasSecurity
	 0 	 hasStorageRoom
	 0 	 price


In [ ]:
print("N/A values for each column: ")
for column in data_SEP.columns.values:
    print("\t", data_SEP[column].isna().sum(), "\t", column)

## Duplikaty

### Duplikaty w danych z miesiąca sierpień

In [27]:
data_AUG.loc[data_ALL.duplicated()]

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price


### Duplikaty w danych z miesiąca wrzesień

In [28]:
data_SEP.loc[data_ALL.duplicated()]

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price


### Duplikaty w danych z miesiąca październik

In [29]:
data_OCT.loc[data_ALL.duplicated()]

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price


### Duplikaty w danych połączonych

In [30]:
duplicates_ALL = data_ALL.loc[data_ALL.duplicated()]
duplicates_ALL.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
18905,71de9eb8a92ac12d7c94315228b70cfb,szczecin,tenement,123.45,4.0,2.0,3.0,NaN,53.432586,14.535685,...,0.321,condominium,brick,NaN,no,yes,no,no,yes,799000
18906,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.073,cooperative,NaN,NaN,no,no,yes,no,no,420000
18909,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
18912,eb15c0a8a7f16a523f5cf5f61de06638,szczecin,blockOfFlats,49.50,2.0,1.0,4.0,1980.0,53.365833,14.661389,...,0.236,cooperative,NaN,low,no,yes,no,no,yes,462000
18922,209a6d2dea99216bdf83a968343cd3b3,szczecin,tenement,61.48,2.0,NaN,3.0,1900.0,53.491508,14.606985,...,1.521,condominium,brick,premium,no,no,no,no,yes,390000


In [31]:
len(duplicates_ALL)

17360

Duplikaty występują jedynie po złączeniu danych z poszczególnych miesięcy. Może to ozanczać, że poszczególne oferty zostały wystawione kilkukrotnie w kolejnych miesiącach lub oferty te były aktywne na przestrzeni kilku miesięcy.

## Liczebności przykładów z poszczególnych klas

In [32]:
# TODO: Co w tym miejscu opisać dla dyskretyzacji

In [33]:
data_SEP["price"].value_counts()

599000     255
699000     238
799000     186
499000     183
650000     163
          ... 
384899       1
894374       1
619999       1
1284000      1
299900       1
Name: price, Length: 2555, dtype: int64

## Współczynnik IR (imbalanced ratio)

## Mapa korelacji atrybutów

## Weryfikacja zasadności pozostawienia/usunięcia poszczególnych atrybutów na podstawie wiedzy dziedzinowej

## Wnioski

# Przygotowanie danych

## Przypisanie miesiąca do poszczególnych ofert na podstawie pliku, z którego pochodzą dane

## Usunięcie duplikatów